# Summarization Model Demonstration

Notebook should be run in our poetry environment.

In [21]:
import os
from pathlib import Path

from src.data.prepare_data import Data, Tokens
from src.inference import batch_predict, interactive_predict, load_model_and_data
from src.model.encoder_decoder import EncoderDecoder
from src.model.hparams import config
from src.utils.device import setup_device
from src.utils.shared_embedding import SharedEmbedding

# С предобученными эмбедингами

In [22]:
DEVICE = setup_device()
MODEL_PATH = str(Path(os.getcwd()).parent / "model/model-30_epochs-without_unk_and_punctuation_all_data.pt")
EMBEDDING_PATH = str(Path(os.getcwd()).parent / "embeddings/navec_hudlit_v1_12B_500K_300d_100q.tar")
DATA_PATH = str(Path(os.getcwd()).parent / "data/news.csv")

try:
    model, data = load_model_and_data(MODEL_PATH, device=DEVICE, embedding_path=EMBEDDING_PATH, data_path=DATA_PATH)
except Exception as e:
    print(f"Error loading model: {e}")
    model, data = None, None

C:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\ml-hw3-6sem-STruoplB-py3.12\Lib\site-packages\gensim\models\keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(
Creating datasets: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74278/74278 [00:17<00:00, 4245.88it/s]
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train size = 63136
2025-05-27 20:34:15,021 - prepare_data - INFO - Train 

In [23]:
test_texts = [
    "Пластик произвел революцию в пищевой упаковке",
    "Ученые обнаружили микрочастицы пластика в чайных пакетиках",
    "Новые исследования показывают опасность нанопластика",
    "В Москве сегодня хорошая погода, светит солнце и температура около 25 градусов.",
    "Книга рассказывает о приключениях молодого волшебника в школе магии.",
    "Президент объявил о новых экономических реформах для поддержки малого бизнеса.",
    "Команда выиграла чемпионат, показав отличную игру на протяжении всего сезона.",
    "Концерт привлек тысячи зрителей и стал одним из самых ярких событий года.",
    "Ученые разрабатывают новые методы лечения рака, основанные на последних достижениях в генетике.",
    "Компания представила новую модель смартфона с улучшенными характеристиками и функциями.",
]

if model and data:
    output_file = "predictions_pretrained_embedding.json"
    try:
        predictions, results = batch_predict(model, data, test_texts, device=DEVICE, output_file=output_file)

        print("Batch Predictions:")
        for i, (text, pred) in enumerate(zip(test_texts, predictions)):
            print(f"\n--- Example {i+1} ---")
            print(f"Input: {text}")
            print(f"Prediction: {pred}")

        print(f"\nPredictions saved to {output_file}")
    except Exception as e:
        print(f"Error during batch prediction: {e}")
else:
    print("Model not loaded, skipping batch prediction.")

Batch Predictions:

--- Example 1 ---
Input: Пластик произвел революцию в пищевой упаковке
Prediction: <unk> сша снизил цены на нефть

--- Example 2 ---
Input: Ученые обнаружили микрочастицы пластика в чайных пакетиках
Prediction: индекс nasdaq вырос на <unk>

--- Example 3 ---
Input: Новые исследования показывают опасность нанопластика
Prediction: в сети появился трейлер нового ледникового периода <unk>

--- Example 4 ---
Input: В Москве сегодня хорошая погода, светит солнце и температура около 25 градусов.
Prediction: в москве нашли <unk> <unk>

--- Example 5 ---
Input: Книга рассказывает о приключениях молодого волшебника в школе магии.
Prediction: в сети появился трейлер нового ледникового периода <unk>

--- Example 6 ---
Input: Президент объявил о новых экономических реформах для поддержки малого бизнеса.
Prediction: в сети появился трейлер нового ледникового периода <unk>

--- Example 7 ---
Input: Команда выиграла чемпионат, показав отличную игру на протяжении всего сезона.
Predi

In [24]:
# Interactive prediction
if model and data:
    try:
        interactive_predict(model, data, DEVICE)
    except Exception as e:
        print(f"Error during interactive prediction: {e}")
else:
    print("Model not loaded, skipping interactive prediction.")


Enter text to summarize (or commands):
- 'debug' to toggle verbose mode
- 'quit' to exit


Exiting...


# С дефолтными эмбедингами

In [25]:
DEVICE = setup_device()
MODEL_PATH = str(
    Path(os.getcwd()).parent / "model/model-30_epochs-usual_embeddings_without_unk_and_punctuation_all_data.pt"
)
DATA_PATH = str(Path(os.getcwd()).parent / "data/news.csv")

try:
    # Initialize data objects
    data = Data()
    train_iter, test_iter = data.init_dataset(
        csv_path=DATA_PATH,
        batch_sizes=(config["train_batch_size"], config["test_batch_size"]),
        split_ratio=config["data_split_ratio"],
    )

    # Initialize SharedEmbedding
    vocab_size = len(data.word_field.vocab)
    d_model = config["d_model"]
    pad_idx = data.word_field.vocab.stoi[Tokens.PAD.value]
    unk_idx = data.word_field.vocab.stoi[Tokens.UNK.value]
    shared_embedding = SharedEmbedding(vocab_size, d_model, pad_idx)

    # Initialize model
    model = EncoderDecoder(
        target_vocab_size=vocab_size,
        shared_embedding=shared_embedding,
        d_model=d_model,
        d_ff=config["d_ff"],
        blocks_count=config["blocks_count"],
        heads_count=config["heads_count"],
        dropout_rate=config["dropout_rate"],
    ).to(DEVICE)

except Exception as e:
    print(f"Error loading model: {e}")
    model, data = None, None

Creating datasets: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74278/74278 [00:05<00:00, 14204.03it/s]
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Train size = 73535
2025-05-27 20:40:37,668 - prepare_data - INFO - Test size = 743
2025-05-27 20:40:37,668 - prepare_data - INFO - Test size = 743
2025-05-27 20:40:37,668 - prepare_data - 

In [26]:
test_texts = [
    "Пластик произвел революцию в пищевой упаковке",
    "Ученые обнаружили микрочастицы пластика в чайных пакетиках",
    "Новые исследования показывают опасность нанопластика",
    "В Москве сегодня хорошая погода, светит солнце и температура около 25 градусов.",
    "Книга рассказывает о приключениях молодого волшебника в школе магии.",
    "Президент объявил о новых экономических реформах для поддержки малого бизнеса.",
    "Команда выиграла чемпионат, показав отличную игру на протяжении всего сезона.",
    "Концерт привлек тысячи зрителей и стал одним из самых ярких событий года.",
    "Ученые разрабатывают новые методы лечения рака, основанные на последних достижениях в генетике.",
    "Компания представила новую модель смартфона с улучшенными характеристиками и функциями.",
]

if model and data:
    output_file = "predictions_usual_embedding.json"
    try:
        predictions, results = batch_predict(model, data, test_texts, device=DEVICE, output_file=output_file)

        print("Batch Predictions:")
        for i, (text, pred) in enumerate(zip(test_texts, predictions)):
            print(f"\n--- Example {i+1} ---")
            print(f"Input: {text}")
            print(f"Prediction: {pred}")

        print(f"\nPredictions saved to {output_file}")
    except Exception as e:
        print(f"Error during batch prediction: {e}")
else:
    print("Model not loaded, skipping batch prediction.")

Batch Predictions:

--- Example 1 ---
Input: Пластик произвел революцию в пищевой упаковке
Prediction: намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный стула стула стула намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный намеченный

In [ ]:
# Interactive prediction
if model and data:
    try:
        interactive_predict(model, data, DEVICE)
    except Exception as e:
        print(f"Error during interactive prediction: {e}")
else:
    print("Model not loaded, skipping interactive prediction.")


Enter text to summarize (or commands):
- 'debug' to toggle verbose mode
- 'quit' to exit

